In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib
#!pip install apache-beam
!pip install tfds-nightly

     |████████████████████████████████| 3.8MB 5.7MB/s 


conveting yolo to csv code before building tfds

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import pandas as pd
import csv
from PIL import Image
import os, sys
shape = 416

In [ ]:
import PIL
import os
import os.path
from PIL import Image

f = r'/content/drive/Shareddrives/EEC193ATrackC/frames'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((shape,shape))
    width, height = img.size
    img.save(f_img)

now we conver the test labels from yolo to csv

Now that we have the nesesary format input we move on to create 

In [ ]:
%cd /content

/content


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
!tfds new Grapes #--overwrite  # Create the new dataset files inside rfcx_tfds/
# [...] Modify the dataset files as you want


2021-03-10 00:17:03.621895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Dataset generated at /content/Grapes
You can start searching `TODO(Grapes)` to complete the implementation.
Please check https://www.tensorflow.org/datasets/add_dataset for additional details.


In [ ]:
%cd /content/Grapes
!tfds build #--max_examples_per_split # Generate the dataSet in the current directory

Streaming output truncated to the last 5000 lines.
Generating train examples...: 23248 examples [5:03:07,  1.39 examples/s]
Generating train examples...: 23249 examples [5:03:08,  1.41 examples/s]
Generating train examples...: 23250 examples [5:03:08,  1.38 examples/s]
Generating train examples...: 23251 examples [5:03:09,  1.41 examples/s]
Generating train examples...: 23252 examples [5:03:10,  1.36 examples/s]
Generating train examples...: 23253 examples [5:03:11,  1.25 examples/s]
Generating train examples...: 23254 examples [5:03:12,  1.28 examples/s]
Generating train examples...: 23255 examples [5:03:12,  1.33 examples/s]
Generating train examples...: 23256 examples [5:03:13,  1.31 examples/s]
Generating train examples...: 23257 examples [5:03:14,  1.33 examples/s]
Generating train examples...: 23258 examples [5:03:14,  1.38 examples/s]
Generating train examples...: 23259 examples [5:03:15,  1.39 examples/s]
Generating train examples...: 23260 examples [5:03:16,  1.39 examples/s]


In [ ]:
#load the data set(just generated)
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import Grapes

ModuleNotFoundError: ignored

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
#just padding so they are all the same size
ds, ds_info = tfds.load(
    'grapes',
    split='train',
    shuffle_files=True,
    as_supervised=True,  # returns `(img, label)` instead of dict(image=, ...)
    with_info=True, 
)
print(ds)
#print(ds_info)
fig = tfds.show_examples(ds, ds_info)



#build trainning pipeline- will use this for the model training
ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds = ds.cache()
# For true randomness, we set the shuffle buffer to the full dataset size.
ds = ds.shuffle(ds_info.splits['train'].num_examples)
# Batch after shuffling to get unique batches at each epoch.
ds = ds.batch(1)
ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
print(ds)
#print(ds_info)


#using this code to test the pipiline and if the data is being loaded correctly
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(416, 416,3)), #need to change thse values and its fix
  tf.keras.layers.Dense(4,activation='relu'),
  tf.keras.layers.Dense(1)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds,
    epochs=6,
    #validation_data=ds,
)


#storing the data on a Google Cloud Storage //buket

DatasetNotFoundError: ignored